<a href="https://colab.research.google.com/github/Antara-Basu/Design-and-Testing-Entropy-Sources-using-NIST-SP-800-90B-Standard/blob/main/Entropy_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Entropy Estimation for  IID and Non-IID Data


For IID data and non-IID data, the following estimators shall be calculated on the outputs of the noise source and the minimum of all the estimates is taken as the entropy assessment of the entropy source for this Recommendation

1)The Most Common Value Estimate


In [1]:
import math

def most_freq(s):

  """
  This function calculates the frequency of the most common element occuring in input s.
  @input: s: input data
  @output: max_count: frequency of the most common element of the input s.

  """

  count = {}
  for i in s:
    if count.get(i):
      count[i] += 1
    else:
      count[i] = 1
  max_count = max(count.values())
  return max_count

def most_common_value_estimate(s):

  """
  This method first finds the proportion 𝑝̂ of the most common value in the input dataset, and then constructs a confidence interval for this proportion. The upper bound of the confidence interval is used to estimate the min-entropy per sample of the source.
  @input: s: input data
  @output: min_entropy: min_entropy of the input s.
  """

  L = len(s)
  p = most_freq(s)/L #Step 1: Find the proportion of the most common value p in the dataset.
  p_dash = min(1, p + 2.576*math.sqrt((p*(1-p))/(L-1))) #Step 2: Calculate an upper bound on the probability of the most common value p_dash as
  min_entropy = -math.log(p_dash, 2) #Step 3: The estimated min-entropy
  return p, p_dash, min_entropy

s = (0, 1, 1, 2, 0, 1, 2, 2, 0, 1, 0, 1, 1, 0, 2, 2, 1, 0, 2, 1)
common_value = most_common_value_estimate(s)
print(common_value)

(0.4, 0.6895174060761333, 0.5363411235066873)


2)The Markov Estimate

In [2]:
import math

def count(s, character):# This function counts the occurence of the given character in s.
    occ = 0
    for c in s:
        if c == character:
            occ += 1
    return occ

def occurence(s, first, second):

  """
  In this function 2 arguments are passed in the function.
  If these two characters are consecutive in s then the count is incremented.
  And it will provide us with how many times they are occuring consecutively.
  """
  c = 0
  for i in range(len(s)-1):
    if s[i] == first and s[i+1] == second:
      c += 1
  return c

def occurence1(s, first, second, third):
  """
  In this function 3 arguments are passed in the function.
  If these three characters are consecutive in s then the count is incremented.
  And it will provide us with how many times they are occuring consecutively.
  """
  c = 0
  for i in range(len(s)-1):
    if s[i] == first and s[i+1] == second and s[i+2] == third:
      c += 1
  return c


def markov_estimate(s):
  """
  This function calculates min_entropy of the input s by markov_estimate.
  @input: s: input data
  @output:min_entropy : min_entropy of the input s
  """

  #STEP 1: p0 and p1 are  calculated using given equation.
  cnt0 = count(s, 0) # count function is used here for calculating the number of 0 in s.
  p0 = cnt0/len(s) # p0 is probability of 0 in s.
  p1 = 1 - p0 # p1 is probability of 1 in s.
  P = {}

  #STEP 2: p00, p11, p10, p01 are calculated here using count and occurence function.
  cnt00 = occurence(s, 0, 0) #occurence function is used to calculate the count of 00 in s.
  cnt01 = occurence(s, 0, 1)
  cnt10 = occurence(s, 1, 0)
  cnt11 = occurence(s, 1, 1)
  p00  = (cnt00  / (cnt00 + cnt01)) #p00 is probability of 00 in s, calculated using count of 00 and 01.
  p01  = (cnt01 / (cnt01 + cnt00 ))
  p10  = (cnt10 / (cnt10 + cnt11))
  p11  = (cnt11 / (cnt11 + cnt10 ))

  p_seq = [0.0 for x in range(6)] #STEP 3: The probability of the most likely sequence of outputs of length 128, as calculated below:
  p_seq[0] = p0 * (p00**127)
  p_seq[1] = p0 * pow(p01,64) * pow(p10,63)
  p_seq[2] = p0 * p01 * (pow(p11,126))
  p_seq[3] = p1 * p10 * (pow(p00,126))
  p_seq[4] = p1 * pow(p10,64) * pow(p01,63)
  p_seq[5] = p1 * pow(p11,127)


  p_max = max(p_seq) #STEP 4 : Let p_max be the maximum of the probabilities in the table given above
  min_entropy = -math.log(p_max,2)/128.0 #Step 5: The min-entropy estimate is the negative logarithm of the probability of the most likely sequence of outputs p_max
  if min_entropy > 1.0:
    min_entropy = 1.0

  return  min_entropy

s = (1,0,0,0,1,1,1,0,0,1,0,1,0,1,0,1,1,1,0,0,1,1,0,0,0,1,1,1,0,0,1,0,1,0,1,0,1,1,1,0)
min_entropy = markov_estimate(s)
print(min_entropy)


0.7606360062539939


3) The Compression Estimate

In [ ]:
import math
"This function converts binary number to integer"
def bits_to_int(bits):
  theint = 0
  for i in range(len(bits)):
    theint = (theint << 1) + bits[i]
  return theint

def F(z,t,u):
  if u < t:
    return (z**2.0)*((1.0-z)**(u-1.0))
  if u == t:
    return z*((1.0-z)**(t-1.0))

def G(z, v, d, L):
  G_sum = 0.0
  st = [math.log(u, 2.0) * ((1.0-z)**(u-1.0)) for u in range((d+1), v+d+1)]
  G_sum = v*z*z * sum([math.log(u, 2.0) * ((1.0-z)**(u-1.0)) for u in range(1,(d+1))])
  G_sum += z*z * sum([(v-t-1) * st[t] for t in range(v-1)])
  G_sum += z * sum(st)
  return G_sum/v

def compression_estimate(bits, d=4):
  """
  This function calculates min_entropy of the input bits by compression_estimate.
  Given a dataset from the noise source, the samples are first partitioned into two disjoint groups.
  The first group serves as the dictionary for the compression algorithm; the second group is used
  as the test group. The compression values are calculated over the test group to determine the mean,
  which is the Maurer statistic. Using the same method as the collision estimate, the probability
  distribution that has the minimum possible entropy for the calculated Maurer statistic is
  determined. For this distribution, the entropy per sample is calculated as the lower bound on the
  entropy that is present.
  @input:bits : input elements
  @output:min_entropy : min_entropy of the input bits
  """

  L = len(bits)
  print("COMPRESSION Test")
  #STEP 1: create a new list s_dash = [s_dash1, s_dash2,.....,s_dash[L/b]] by dividing L into non-overlapping b-bit blocks. If L is not a multiple of b, discard the extra data.
  b = 6
  blocks = L//b
  s_dash = [0,]+[bits_to_int(bits[b*i:b*(i+1)]) for i in range(blocks)]

  #STEP 2: Partition the dataset s_dash, into two disjoint groups. These two groups will form the dictionary_data and the testing_data.
  #STEP 2a: Create the dictionary from the first d = 4 elements of s_dash, [s_dash1,s_dash2,.....,s_dashd]
  dictionary_data = s_dash[1:d+1]
  v = blocks-d
  #STEP 2b: Use the remaining v = ⌊L/b⌋ − d observations i,e, 4, (s_dash𝑑+1′ , … , s_dash⌊L/b⌋), for testing
  testing_data=s_dash[d+1:]


  print("   v                   ",v)

  #STEP 3: Initialize the dictionary to an all zero array of size 2**b
  dictionary = [0 for i in range((2**b)+1)] # Make it 1 bigger and leave the zero element dangling
                                              # so the indexes match the spec which uses 1 based arrays.
  for i in range(1,d+1):
      dictionary[s_dash[i]]=i

  #STEP 4: Run the test data against the dictionary_data created in Step 2.
  #STEP 4a: Let D be a list of length v.
  #STEP 4b: For i from d + 1 to ⌊L/b⌋
  D = [0,]+[0 for i in range(v)]
  for i in range(d+1,blocks+1):
    if dictionary[s_dash[i]] != 0: #steps 4b(1)
      D[i-d] = i-dictionary[s_dash[i]]
      dictionary[s_dash[i]] = i
    if dictionary[s_dash[i]] == 0: #steps 4b(2)
      dictionary[s_dash[i]] = i
      D[i-d] = i

  # STEP 5: Calculating sample mean, x_bar

  add = 0.0
  for i in range(1,v+1):
    add += math.log(D[i],2)
    X_bar = add/v

  print("   X_bar               ",X_bar)

  c = 0.5907

  d_add = 0.0 # declaration of a variable for calculating standard deviation
  for i in range(1,v+1):
    d_add += (math.log(D[i],2)**2)
  d_add = d_add/(v-1.0)
  d_add = d_add - (X_bar**2)
  stand_devi = c * math.sqrt(d_add) # calculating standard deviation

  print("   stand_devi          ",stand_devi)

  # STEP 6: Compute the lower-bound of the confidence interval for the mean, based on a normal distribution

  X_bar_dash = X_bar - ((2.576*stand_devi)/math.sqrt(v))
  print("   X_bar_dash         ",X_bar_dash)

  # STEP 7: Using a binary search, solve for the parameter P

  p_min = 2.0 ** -b  # binary search bounds
  p_max = 1.0
  p_mid = (p_min+p_max)/2.0
  iterations = 1000
  iteration = 0

  found = False
  while (iteration < iterations):
    q = (1.0-p_mid)/((2.0**b)-1.0)
    candidate = G(p_mid,v,d,L) + (((2.0**b)-1.0)*G(q,v,d,L))

    if abs(candidate -X_bar_dash) < 0.00000000001:
      found = True
      break
    elif candidate > X_bar_dash:
      p_min = p_mid
      p_mid = (p_min+p_max)/2.0
    elif candidate < X_bar_dash:
      p_max = p_mid
      p_mid = (p_min+p_max)/2.0

      iteration += 1

  print("   p          =",p_mid)
  # STEP 8: If the binary search yields a solution, then the min-entropy is the negative logarithm of the parameter p , otherwise its 1.
  if found:
    min_entropy = -math.log(p_mid,2)/b
    print("   min_entropy =",min_entropy)
    return min_entropy
  else:
    min_entropy = 1.0
    print("   min_entropy = 1.0")
    return min_entropy

bits = (1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1 ,1, 1, 0, 0, 0, 1, 1)
min_entropy = compression_estimate(bits)


COMPRESSION Test
   v                    4
   X_bar                2.6304001099309313
   stand_devi           0.9073768771997716
   X_bar_dash          1.4616986920976256
   p          = 0.5714846826669486
   min_entropy = 0.13453554400886117
